# Крестики-нолики

Правила просты. Есть доска NxN. Два игрока попеременно ставять "Х" или "О" в клетки этой доски. Как только кто-то составит линию по горизонтали, вертикали или диагонали из M своих значков, он объявляется победителем. Если до заполнения доски такого не происходит, то объявляется ничья.

## Тривиальная реализация

In [1]:
# Параметры нашей доски
N = 3
M = 3

# Сама доска
board = [
    [None for _ in range(N)]
    for _ in range(N)
]


def render(N, board):
    """
        Функция для красивой распечатки доски 
    """
    print(" _"*N)
    for row in board:
        print("|", end="")
        for cell in row:
            c = "_" if cell is None else cell
            print("%s|" % c, end="")
        print()
    print()
  
def who_win(N, M, board):
    """
        Функция для проверки, кто победил на доске.
        
        Возвращется "X" или "O", если победил какой-то игрок
        В противном случае возвращает  None
    """
    
    # Строки с помощью которых будет искать подряд идущие символы
    x_win = "X"*M
    o_win = "O"*M
    
    # Склеиваем строки доски в обычные строки питона
    rows = [
        "".join(
            [c if c is not None else "_" for c in row]
        )
        for row in board
    ]
    
    # Проверяем, встречается ли в строке подряд идущие символы X или О
    if any([x_win in line for line in rows]):
        return "X"

    if any([o_win in line for line in rows]):
        return "O"
    
    # Склеиваем столбцы доски в обычные строки питона
    cols = [
        "".join(
            [board[y][x] if board[y][x] is not None else "_" for y in range(N)]
        )
        for x in range(N)
    ]
    
    if any([x_win in line for line in cols]):
        return "X"

    if any([o_win in line for line in cols]):
        return "O"
    
    # Работы с диагоналями пользуемся тем фактом, что разница координат 
    # для элементов одной диагонали является инвариантом
    
    # Склеиваем главные диагонали доски в обычные строки питона
    diag1 = {}
    for y in range(N):
        for x in range(N):
            c = board[y][x]
            d = x - y
            
            diag1[d] = diag1.get(d, []) + [(x, y)]
            
    # Склеиваем побочные диагонали доски в обычные строки питона
    diag2 = {}
    for y in range(N):
        for x in range(N):
            c = board[y][x]
            d = (N - 1 - x) - y
            
            diag2[d] = diag2.get(d, []) + [(x, y)]
            
    diag1 = [
        "".join(
            [board[p[1]][p[0]] if board[p[1]][p[0]] is not None else "_" for p in positions]
        )
        for positions in diag1.values()
    ]
    diag2 = [
        "".join(
            [board[p[1]][p[0]] if board[p[1]][p[0]] is not None else "_" for p in positions]
        )
        for positions in diag2.values()
    ]
    
    if any([x_win in line for line in diag1]):
        return "X"

    if any([o_win in line for line in diag1]):
        return "O"
    
    if any([x_win in line for line in diag2]):
        return "X"

    if any([o_win in line for line in diag2]):
        return "O"

    # Победителя нет
    return None
    
# Отслеживаем ход игры
ok = True
winner = None

# Чей сейчас ход
player = "X"

while ok:
    print("-"*80)
    render(N, board)
    
    # Собираем координаты всех пустых клеток доски
    positions = []
    for y, row in enumerate(board):
        for x, cell in enumerate(row):
            if cell is None:
                positions.append((x, y))
    
    # Игра закончена, если делать больше нечего
    if len(positions) == 0:
        winner = None
        ok = False
        continue
        
    print("Current player:", player)
    print("Select positions: ")
    for i, p in enumerate(positions):
        # Выводим нумерацию позиций пользователю с 1, а не нуля
        print(f'{i + 1: 2d}) [x={p[0] + 1:2d} | y={p[1] + 1:2d}]')
    
    try:
        p_idx = int(input())
    except:
        print("Try again")
        continue
        
    # Если кривой ввод, считаем, что пользователь сдался
    if not (0 < p_idx <= len(positions)):
        winner = "X" if player == "O" else "O"
        ok = False
        continue

    # помним, что пользователь видит нумерацию с 1, поэтому отнимает 1,
    # чтобы она была с 0
    p = positions[p_idx - 1]
    board[p[1]][p[0]] = player
    
    winner = who_win(N, M, board)
    if winner is not None:
        ok = False
    
    # Смена игрока
    # можно было так 
    # player = "X" if player == "O" else "O"
    if player == "X":
        player = "O"
    else:
        player = "X"
    
# Выводим состояния
print("-"*80)
render(N, board)
print("Winner: ", winner)

--------------------------------------------------------------------------------
 _ _ _
|_|_|_|
|_|_|_|
|_|_|_|

Current player: X
Select positions: 
 1) [x= 1 | y= 1]
 2) [x= 2 | y= 1]
 3) [x= 3 | y= 1]
 4) [x= 1 | y= 2]
 5) [x= 2 | y= 2]
 6) [x= 3 | y= 2]
 7) [x= 1 | y= 3]
 8) [x= 2 | y= 3]
 9) [x= 3 | y= 3]


 -1


--------------------------------------------------------------------------------
 _ _ _
|_|_|_|
|_|_|_|
|_|_|_|

Winner:  O


# Щепотка ООП

В реализации выше есть множество недостатков
* код запутан и трудно читаем
* одни задачи переплетены с другими
* фактически нужно писать с нуля, если меняется окружения запуска
* сложно менять поведения игроков (добавить ИИ)


Чтобы избавиться от этих проблем, удобно переписать код используя концепции ООП. Обычно все начинается с дизайна, в этом активно помогает UML

# Board

Первое, что мы видим игра к крестики-нолики - это доска. Так что первым этапом опишем доску, как единный объект. Этим мы скроем конкретное представление доски в терминах питона и облегчим работу с самой доской

In [1]:
from abc import ABC, abstractmethod

class Board(ABC):
    """
        Интерфейс доски для игры. У каждой клетки есть своя координата.
        
        x - по горизонтали слева направо
        y - по вертикали сверху вниз
        
        Координата верхней левой клетки (1, 1)
    """
    
    @abstractmethod
    def clone(self):
        """
            Метод для создания копии доски
        """
        ...
    
    @abstractmethod
    def get_cell(self, x, y):
        """
            Получить значение в клетке по координатам
        """
        ...
        
    @abstractmethod
    def set_cell(self, x, y, cell):
        """
            Выставить значение в клетке по координатам
        """
        ...
      
    @abstractmethod
    def itercells(self):
        """
            Пройтись в клеткам по порядку - слева направо, сверху вниз
            
            То есть сначала выводится первая строка, потом вторая и т.д.
        """
        ...
                
    
    @abstractmethod
    def iterrows(self):
        """
            Пройтись по строкам сверху вниз
        """
        ...
       
    @abstractmethod
    def itercols(self):
        """
            Пройтись по столбцам слева направо
        """
        ...
        
    @abstractmethod
    def iterdiag1(self):
        """
            Пройтись по главным диагоналям
        """
        ...

    @abstractmethod
    def iterdiag2(self):
        """
            Пройтись по побочным диагоналям
        """
        ...
            

In [2]:
class SquareBoard(Board):
    """
        Квадратная доска
    """
    
    def __init__(self, N):
        self.__N = N
        self.__board = [
            [None for _ in range(self.__N)]
            for _ in range(self.__N)
        ]
        
    def clone(self):
        import copy
        b = SquareBoard(self.__N)
        b.__board = copy.deepcopy(self.__board)
        return b
    
    def get_cell(self, x, y):
        if not(0 < x <= self.__N) or not(0 < x <= self.__N):
            raise Exception("Too big or too low x/y")
            
        return self.__board[y-1][x-1]
        
    def set_cell(self, x, y, cell):
        if not(0 < x <= self.__N) or not(0 < y <= self.__N):
            raise Exception("Too big or too low x/y")
            
        if cell not in [None, "X", "O"]:
            raise Exception("Not valid value")

            
        self.__board[y-1][x-1] = cell
        
    def itercells(self):
        for ry in range(self.__N):
            for rx in range(self.__N):
                yield ((rx+1, ry+1), self.__board[ry][rx])
                
                
    def iterrows(self):
        for ry, row in enumerate(self.__board):
            yield [ ((rx+1, ry+1), value) for rx, value in enumerate(row) ]
        
    def itercols(self):
        for rx in range(self.__N):
            yield [ ((rx+1, ry+1), self.__board[ry][rx]) for ry in range(self.__N) ]
            
    def iterdiag1(self):
        diag = {}
        for (x, y), _ in self.itercells():
            d = x - y
            diag[d] = diag.get(d, []) + [(x, y)]
        
        for k in sorted(diag):
            yield [ ((x, y), self.__board[y-1][x-1]) for x, y in diag[k] ]

    def iterdiag2(self):
        diag = {}
        for (rx, ry), _ in self.itercells():
            d = (self.__N - 1 - rx) - ry
            diag[d] = diag.get(d, []) + [(rx, ry)]
        
        for k in sorted(diag):
            yield [ ((x, y), self.__board[y-1][x-1]) for x, y in diag[k] ]
                 
    def __str__(self):
        s =  "\n     " + " ".join(["%3d" % (col+1) for col in range(self.__N)]) + "\n" 
        s += "     " + "--- " * self.__N + "\n"
        for n, row in enumerate(self.__board):
            s += " %2d |" % (n+1)  + "".join([
                " %s |" % (v if v is not None else " ") for v in row
            ]) + "\n" 
            s += "     " + "--- " * self.__N + "\n"
        s += "\n"
        
        return s
    
    def __repr__(self):
        return self.__str__()
    
    
    
b = SquareBoard(4)
b.set_cell(1, 1, "X")
b.set_cell(3, 3, "O")
nb = b.clone()
nb.set_cell(4, 1, "X")
print("Cloned:", nb)

print("Original:", b)
print()

Cloned: 
       1   2   3   4
     --- --- --- --- 
  1 | X |   |   | X |
     --- --- --- --- 
  2 |   |   |   |   |
     --- --- --- --- 
  3 |   |   | O |   |
     --- --- --- --- 
  4 |   |   |   |   |
     --- --- --- --- 


Original: 
       1   2   3   4
     --- --- --- --- 
  1 | X |   |   |   |
     --- --- --- --- 
  2 |   |   |   |   |
     --- --- --- --- 
  3 |   |   | O |   |
     --- --- --- --- 
  4 |   |   |   |   |
     --- --- --- --- 





In [3]:
print(b)
print()
for data in b.itercells():
    print(data)
print()


       1   2   3   4
     --- --- --- --- 
  1 | X |   |   |   |
     --- --- --- --- 
  2 |   |   |   |   |
     --- --- --- --- 
  3 |   |   | O |   |
     --- --- --- --- 
  4 |   |   |   |   |
     --- --- --- --- 



((1, 1), 'X')
((2, 1), None)
((3, 1), None)
((4, 1), None)
((1, 2), None)
((2, 2), None)
((3, 2), None)
((4, 2), None)
((1, 3), None)
((2, 3), None)
((3, 3), 'O')
((4, 3), None)
((1, 4), None)
((2, 4), None)
((3, 4), None)
((4, 4), None)



In [4]:
print(b)
print()
for data in b.iterrows():
    print(data)
print()


       1   2   3   4
     --- --- --- --- 
  1 | X |   |   |   |
     --- --- --- --- 
  2 |   |   |   |   |
     --- --- --- --- 
  3 |   |   | O |   |
     --- --- --- --- 
  4 |   |   |   |   |
     --- --- --- --- 



[((1, 1), 'X'), ((2, 1), None), ((3, 1), None), ((4, 1), None)]
[((1, 2), None), ((2, 2), None), ((3, 2), None), ((4, 2), None)]
[((1, 3), None), ((2, 3), None), ((3, 3), 'O'), ((4, 3), None)]
[((1, 4), None), ((2, 4), None), ((3, 4), None), ((4, 4), None)]



In [5]:
print(b)
print()
for data in b.itercols():
    print(data)
print()


       1   2   3   4
     --- --- --- --- 
  1 | X |   |   |   |
     --- --- --- --- 
  2 |   |   |   |   |
     --- --- --- --- 
  3 |   |   | O |   |
     --- --- --- --- 
  4 |   |   |   |   |
     --- --- --- --- 



[((1, 1), 'X'), ((1, 2), None), ((1, 3), None), ((1, 4), None)]
[((2, 1), None), ((2, 2), None), ((2, 3), None), ((2, 4), None)]
[((3, 1), None), ((3, 2), None), ((3, 3), 'O'), ((3, 4), None)]
[((4, 1), None), ((4, 2), None), ((4, 3), None), ((4, 4), None)]



In [6]:
print(b)
print()
for data in b.iterdiag1():
    print(data)
print()


       1   2   3   4
     --- --- --- --- 
  1 | X |   |   |   |
     --- --- --- --- 
  2 |   |   |   |   |
     --- --- --- --- 
  3 |   |   | O |   |
     --- --- --- --- 
  4 |   |   |   |   |
     --- --- --- --- 



[((1, 4), None)]
[((1, 3), None), ((2, 4), None)]
[((1, 2), None), ((2, 3), None), ((3, 4), None)]
[((1, 1), 'X'), ((2, 2), None), ((3, 3), 'O'), ((4, 4), None)]
[((2, 1), None), ((3, 2), None), ((4, 3), None)]
[((3, 1), None), ((4, 2), None)]
[((4, 1), None)]



In [7]:
print(b)
print()
for data in b.iterdiag2():
    print(data)
print()


       1   2   3   4
     --- --- --- --- 
  1 | X |   |   |   |
     --- --- --- --- 
  2 |   |   |   |   |
     --- --- --- --- 
  3 |   |   | O |   |
     --- --- --- --- 
  4 |   |   |   |   |
     --- --- --- --- 



[((4, 4), None)]
[((4, 3), None), ((3, 4), None)]
[((4, 2), None), ((3, 3), 'O'), ((2, 4), None)]
[((4, 1), None), ((3, 2), None), ((2, 3), None), ((1, 4), None)]
[((3, 1), None), ((2, 2), None), ((1, 3), None)]
[((2, 1), None), ((1, 2), None)]
[((1, 1), 'X')]



# State

Помимо доски, у нас есть текущее состояние игры: кто ходит, победил ли кто, доска на которой происходит бой и прочее. Удобно работать с концепцией неизменяемого состояния, когда следующие состояния порождаются предыдущими без их изменения.

In [4]:
class State:
    """
        Описывает текущее состояние игры
    """
    def __init__(self, *, M=3, board=None, player=1, parent=None):
        self.__parent = parent
        self.__player = player
        self.__board = board
        self.__M = M if parent is None else parent.__M
        
    @property
    def parent(self):
        """
            Ссылка на родителя
        """
        return self.__parent
    
    @property
    def board(self):
        """
            Свойствой, которое позволяет обращаться к доске,
            но не позволяет ее сменить (очень дорошая реализация)
        """
        return self.__board.clone()
    
    @property
    def player(self):
        """
            Идентификатор текущего игрок (в нашем случае - это номер)
        """
        return self.__player
            
        
    def clone(self):
        """
            Создание клона
        """
        s = State(
            player=self.__player,
            board=self.__board.clone(), 
            parent=self.__parent)
        
        s.__player = self.__player
        return s
    
    def is_end(self):
        """
            Проверка на то, является ли это состояние конечным
        """
        winner = self.has_winner()
        if winner is not None:
            return True
        
        for _, value in self.__board.itercells():
            if value is None:
                return False
        return True
    
    def has_winner(self):
        """
            Ищем победителя
        """
        x_win = "X"*self.__M
        o_win = "O"*self.__M
        
        rows = list(self.__board.iterrows())
        cols = list(self.__board.itercols())
        diag1 = list(self.__board.iterdiag1())
        diag2 = list(self.__board.iterdiag2())
        
        for checks in [rows, cols, diag1, diag2]:
            for seq in checks:
                line = "".join([c if c is not None else "_" for _, c in seq])
                if x_win in line:
                    return "X"
                if o_win in line:
                    return "O"
                
        return None

    def get_actions(self):
        """
            Для простоты будем считать, что само состояние знает, как 
            породить новые состояния. Конкретное создание новых состояний
            делегируем специальному классу Action
            
            В нормальной реализации нужно вынести в отдельный объект,
            который реализует правила игры.
        """
        if self.is_end():
            return []
        
        actions = []
        for (x, y), cell in self.__board.itercells():
            if cell is None:
                actions.append(
                    PutXOAndNextAction(self, x, y)
                )
        return actions 
    
    def __str__(self):
        s =  f"Current player: [{'X' if self.__player == 1 else 'O'}]\n"
        s += str(self.__board)
        return s
    
    def __repr__(self):
        return self.__str__()
        
class Action(ABC):
    """
        Абстрактный класс, реализующий ход игрока
    """
    def __init__(self, msg):
        self.__msg = msg
        
    @abstractmethod
    def do(self):
        """
            Метод, который из текущего состояния создает новое
            на основе хода игрока
        """
        ...
        
    def msg(self):
        """
            Сообщение для пользователя
        """
        return self.__msg
        

class PutXOAndNextAction(Action):
    def __init__(self, state, x, y):
        super().__init__(
            f'Put [{"X" if state.player == 1 else "O"}] in ({x:2d}, {y:2d})'
        )
        
        self.__state = state
        self.__x = x
        self.__y = y
        
    def do(self):
        board = self.__state.board
        board.set_cell(self.__x, self.__y, "X" if self.__state.player == 1 else "O")
        
        s = State(
            board=board, parent=self.__state,
            player=1 if self.__state.player == 2 else 2,
        )
        return s

In [5]:
s = State(board=SquareBoard(3), M=3)
print(s)
print()
actions = s.get_actions()
for n, a in enumerate(actions):
    print(f'{n+1:2d}) {a.msg()}')
    
act = actions[0]
s = act.do()
print()
print()
print(s)

Current player: [X]

       1   2   3
     --- --- --- 
  1 |   |   |   |
     --- --- --- 
  2 |   |   |   |
     --- --- --- 
  3 |   |   |   |
     --- --- --- 



 1) Put [X] in ( 1,  1)
 2) Put [X] in ( 2,  1)
 3) Put [X] in ( 3,  1)
 4) Put [X] in ( 1,  2)
 5) Put [X] in ( 2,  2)
 6) Put [X] in ( 3,  2)
 7) Put [X] in ( 1,  3)
 8) Put [X] in ( 2,  3)
 9) Put [X] in ( 3,  3)


Current player: [O]

       1   2   3
     --- --- --- 
  1 | X |   |   |
     --- --- --- 
  2 |   |   |   |
     --- --- --- 
  3 |   |   |   |
     --- --- --- 




# Игрок

Игрок тоже является объектом. Удобно объявить интерфейс игрока, от которого мы можем создавать конкретные реализации игроков.

In [6]:
class Player(ABC):
    def __init__(self, name):
        self.__name = name
        
    @property
    def name(self): return self.__name

    @abstractmethod
    def select(state, actions):
        """
            Выбирает действие среди доступных и возвращает его.
            
            Либо возвращает None, если игрок отказался от доступных действий
        """
        ...
        
class MCTSPlayer(Player):
    ...
    
class NetworkPlayer(Player):
    ...

In [7]:
class ConsolePlayer(Player):
    """
        Класс, который реализует взаимодействие с игроком
        посредством терминала
    """
    def select(self, state, actions):
        if len(actions) == 0:
            return None
        
        print("Player - %s" % self.name)
        print(state)
        for n, a in enumerate(actions):
            print(f'{n+1:2d}) {a.msg()}')
            
        print(f'Any other number to exit.')  
            
        while True:
            try:
                result = int(input("Please, enter a number: "))
            except:
                print("Try again")
                continue
            break
            
        return actions[result-1] if 0 < result <= len(actions) else None
          
player = ConsolePlayer("John")
player.select(s, s.get_actions())

Player - John
Current player: [O]

       1   2   3
     --- --- --- 
  1 | X |   |   |
     --- --- --- 
  2 |   |   |   |
     --- --- --- 
  3 |   |   |   |
     --- --- --- 


 1) Put [O] in ( 2,  1)
 2) Put [O] in ( 3,  1)
 3) Put [O] in ( 1,  2)
 4) Put [O] in ( 2,  2)
 5) Put [O] in ( 3,  2)
 6) Put [O] in ( 1,  3)
 7) Put [O] in ( 2,  3)
 8) Put [O] in ( 3,  3)
Any other number to exit.


Please, enter a number:  0


In [8]:
class RandomAIPlayer(Player):
    """
        "Случайный искусственный интеллект"
    """
    def select(self, state, actions):
        import random 
        
        if len(actions) == 0:
            return None
        
        return random.choice(actions)
 
player = RandomAIPlayer("Dummy")
player.select(s, s.get_actions())

In [12]:
# Пример игры двух игроков

s = State(board=SquareBoard(3), M=3)

players = [
    None, 
    RandomAIPlayer("Dummy1"),
    RandomAIPlayer("Dummy2"),
]

while True:
    if s.is_end(): 
        break
        
    actions = s.get_actions()
    act = players[s.player].select(s, actions)
    if act == None:
        break
    s = act.do()
    
print(s)
print("Winner: ", s.has_winner())

Current player: [O]

       1   2   3
     --- --- --- 
  1 | X | X |   |
     --- --- --- 
  2 | X | O | O |
     --- --- --- 
  3 | X | O |   |
     --- --- --- 


Winner:  X


# Игра

Саму игру тоже удобно завернуть в класс.

In [13]:
class Game:
    """
        Реализует саму игру в целом
    """
    def __init__(self, player1, player2, *, board, M):
        self.__players = [None, player1, player2]
        self.__state = State(board=board, M=M)
        
    def start(self):
        while True:
            if s.is_end(): 
                break

            actions = self.__state.get_actions()
            act = players[self.__state.player].select(self.__state, actions)
            if act == None:
                break
            self.__state = act.do()

        print(self.__state)
        print("Winner: ", self.__state.has_winner())

# Модуль

In [14]:
import tictactoe

g = tictactoe.Game(
    tictactoe.RandomAIPlayer("Dummy1"),
    tictactoe.RandomAIPlayer("Dummy2"),
    board=tictactoe.SquareBoard(3), M=3, 
)

g.start()

Current player: [X]

       1   2   3
     --- --- --- 
  1 | O | X | X |
     --- --- --- 
  2 |   | O | O |
     --- --- --- 
  3 | X | X | O |
     --- --- --- 


Winner:  O


# Monte-Carlo Tree Search

Благодаря коду выше, мы теперь можем построить полное дерево состояний игры, то есть все возможные варианты игры. Если такое дерево не очень большое, то в нем достаточно легко найти оптимальные стратегии. На практике же, обычно состояний очень много и хранить такой объект информации не представляется возможным. Примером таким игр являются шахматы, го, рендзю и многие другие. Для обычный крестиков-ноликов - это состовляет 362880 состояний (без учета симметрий).

Появление алгоритмы MCTS позволило хоть как-то оптимально искать в больших деревьях оптимальный путь. Следующим шагом развития этого алгоритма стал AlphaZero.

<img src="https://upload.wikimedia.org/wikipedia/commons/1/1f/Tic-tac-toe-full-game-tree-x-rational.jpg">

Рассмотрим самый классический вариант MCTS основанный на алгоритме Upper Confidence Bound 1 (UCB1).  В этом случае у нас также имеется дерево состояни, но помимо текущего состояния игры, там хранится количество отыгранных игр и количество побед первого и второго игрока.

Поиск по дереву состоит из 4-х этапов.

<img src="https://upload.wikimedia.org/wikipedia/commons/2/21/MCTS-steps.svg">

### Selection

Первый этап - это выбор состояния, которое мы еще не исследовали. Кратко это можно описать следующим образом:

1) Находясь в текущем узле дерева состояния, мы для каждого $i$-го дочернего узла вычисляем
$$
score = \frac{\omega_i}{n_i} + c \sqrt{\frac{\ln N}{n_i}}
$$
где $\omega_i$ - это количество побед (в общем случае сумма выйгрыша) в дочернем узле для текущего игрока, $n_i$ - сколько игр сыграно в дочернем узле. $N$ - сколько игр сыграно в текущем узле, $c$ - константа управляющая поиском. Из теории она равна $\sqrt{2}$, но может выбираться эмпирически. При больших значения - поиск становится шире, при маленьких - глубже.

2) Выбираем узел с максимальным значением score и делаем его текущим.

3) Если в выбранном узле есть дочерние состояния без статистики игр (они фактически еще не добавлены в дерево), то переходим к следующему этапу - расширение. В противном случае переходим обратно к пункту 1.

### Expansion

На данном этапе все просто. Мы выбираем любой недобавленный дочерний узел, затем присоединяем его к нашему дереву.

### Simulation

Третий этап тоже простой. Мы просто доигрываем игру из текущего состояния любым способом. Лучше всего использовать эвристику, но можно и просто совершать случайные ходы. Увеличиваем счетчик побед победившего игрока в присоединенном узле (в общем случае сумма выйгрыша).

### Backpropagation
 
Этот этап немного сложнее. Нам нужно обновить информация во всех родительских узлах. Во всех родительских узлах, вплоть до корня дерева, мы увеличиваем счетчик сыгранных игр на 1 и увеличиваем счетчик победившего игрока на 1 (в общем случае на сумму выйгрыша сыгранной игры).

## Финал

После некоторого числа повтороний вышеописанных этапов, мы просто выбираем в качестве следующего хода узел с наибольшим количеством сыгранных игр (количеством посещений).

# Реализация

In [43]:
from abc import ABC, abstractmethod
import math

class MCTSNode(ABC):
    """
        Абстрактный узел дерева для MCTS
    """
    
    def __init__(self, *, parent=None):
        """
            Создание узла. Узел должен знать только родителя и статистику побед
        """
        # сколько раз посетили
        self.__N = 0
        # ссылка на родителя
        self.__parent = parent
        # статистика побед каждого игрока, ключ - это идентификатор игрока
        self.__score = {}
        # ссылки на дочерние узлы в виде списка
        # None - потомки еще не проверялись
        # Список - если элемент в списке None - потомок не рассматривался, 
        #          в противном случае там будет ссылка на него
        self.__childs = None
        
        
    def best_move(self):
        """
            Получить номер наилучшего хода
            Берем просто самый посещяемый узел относительного текущего
        """
        best = None
        best_n = None
        for n, child in enumerate(self.__childs):
            if best_n is None or child.__N > best_n:
                best = n
                best_n = child.__N
        return best
        
    @abstractmethod
    def player(self):
        """
            Идентификатор текущего игрока
        """
        ...
        
        
    def score(self):
        """
            Вес важности узла согласно UCB1. 
            Узлы с наибольшим весом будут посещаться первыми.
        """
        current = self.__parent.player()
        c = math.sqrt(2)
        return self.__score.get(current, 0)/self.__N + c * math.sqrt(math.log(self.__parent.__N)/self.__N)
        
    def selection(self):
        """
            Эта отбора. Рекурсивно спускаемся по дереву согласно алгоритму.
            Мы должны вернуть узел, подходящий для следующего шага
        """
        
        # если узел не проверялся на детей, то значит это наш кандидат
        if self.__childs is None:
            return self
        
        # в противном случае ищем дочерний узел с максимальным весом
        best = None
        best_score = None
        for child in self.__childs:
            # если нашли несуществуюещего потомка, значит этот узел
            # еще пригоден для расширения
            if child is None:
                return self
            
            score = child.score()
            if best_score is None or score > best_score:
                best = child
                best_score = score
                
        # теперь проблема выбора - это головная боль следуюшего узла
        return best.selection()
    
    @abstractmethod
    def is_end(self):
        """
            Является ли данный узел терминальным (концом игры)
        """
        ...
    
    @abstractmethod
    def number_of_childs(self):
        """
            Сколько потомков может быть у данного узла
        """
        ...
        
    @abstractmethod
    def build_child(self, n):
        """
            Создать n-ый узел-потомок 
        """
        ...
    
    def expansion(self):
        """
            Шаг расширения дерева, мы добавляем узел и возвращаем на него 
            ссылку, чтобы провести в нем игру
        """
        
        # Если это терминальный узел, то расширять нечего, игру будем проводить для него
        if self.is_end():
            return self
            
        # Если не проверялись на детей, то создаем список из непроинициализированных
        # ссылок
        if self.__childs is None:
            self.__childs = [None for _ in range(self.number_of_childs())]
        
        # Ищем первого непроинициализированного потомка
        for n, child in enumerate(self.__childs):
            if child is None:
                child = self.build_child(n)
                self.__childs[n] = child
                return child
            
        # Это по идее никогда не должно произойти
        raise Exception()
    
    @abstractmethod
    def simulate(self):
        """
            Симуляция игры. Данный метод должен возвращать словарь, где ключи - это
            идентификатор игрока, а значение - сколько очков набрал игрок (в нашем случае
            1 - за победу, 0.5 - за ничью, 0 - за провал)
        """
        ...
        
    def backpropagate(self, score):
        """
            Обновляем информацию в текущем узле и родительских узлах
        """
        for p in score:
            self.__score[p] = self.__score.get(p, 0) + score[p]
        self.__N += 1
        
        # если есть родитель, то информируем его
        if self.__parent is not None:
            self.__parent.backpropagate(score)
    
    
def step(root):
    """
        Один шаг нашего алгоритма, нужно повторить N раз
    """
    node = root.selection()
    child = node.expansion()
    score = child.simulate()
    child.backpropagate(score)

# MCTS for TicTacToe

In [44]:
import random

class TicTacToeNode(MCTSNode):
    """
        Заполняем недостающие части нашего узла
    """
    def __init__(self, state, *, parent=None):
        super().__init__(parent=parent)
        self.__state = state

    def player(self):
        return self.__state.player
    
    def is_end(self):
        return self.__state.is_end()
    
    def number_of_childs(self):
        return len(self.__state.get_actions())
    
    def build_child(self, n):
        state = self.__state.get_actions()[n].do()
        return TicTacToeNode(state, parent=self)
    
    def simulate(self):
        # случайная игра
        s = self.__state
        while True:
            if s.is_end(): 
                break

            actions = s.get_actions()
            act = random.choice(actions)
            s = act.do()
        winner = s.has_winner()
        if winner == "X":
            return {1: 1.0, 2: 0.0}
        if winner == "O":
            return {1: 0.0, 2: 1.0}
        if winner is None:
            return {1: 0.5, 2: 0.5}
        
        raise Exception()
        
root = TicTacToeNode(State(board=SquareBoard(3), M=3))

for _ in range(100):
    step(root)
    
root.best_move()

8

In [45]:
import tictactoe

class MCTSPlayer(tictactoe.Player):
    """
        Искусственный интеллект с настраиваемой сложностью
    """
    def __init__(self, name, difficult=1000):
        super().__init__(name)
        self.__difficult = difficult
        
    def select(self, state, actions):
        root = TicTacToeNode(state)
        
        # запускаем шаги MCTS
        for _ in range(self.__difficult):
            step(root)
        
        # выбираем наилучший
        return actions[root.best_move()]

In [51]:
import tictactoe

g = tictactoe.Game(
    MCTSPlayer("Dummy1"),
    MCTSPlayer("Dummy2"),
    board=tictactoe.SquareBoard(4), M=3, 
)

g.start()

Current player: [X]

       1   2   3   4
     --- --- --- --- 
  1 | X |   | X |   |
     --- --- --- --- 
  2 |   | O |   |   |
     --- --- --- --- 
  3 |   | X | O |   |
     --- --- --- --- 
  4 |   |   |   | O |
     --- --- --- --- 


Winner:  O


In [9]:
import tictactoe

root = TicTacToeNode(tictactoe.State(board=tictactoe.SquareBoard(3), M=3))
for _ in range(1000):
    mcts.step(root)
print(root)
move = root.best_move()
print(move)
root.childs[move]._TicTacToeNode__state

[687.50/312.50] | 1000 |   9/  9
4


Current player: [O]

       1   2   3
     --- --- --- 
  1 |   |   |   |
     --- --- --- 
  2 |   | X |   |
     --- --- --- 
  3 |   |   |   |
     --- --- --- 


# Что дальше?

- оптимизация симуляции игры (эвристики или заменить нейросетью)
- кэширование тяжелых вычислений 
- повторное использование узлов дерева, чтобы не считать его каждый раз заново
- добавление предварительной оценки состояний игры (эвристики или вычислить нейросетью)